In [1]:
import numpy
import numpy as np
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, execute, BasicAer
from qiskit.extensions.standard import RCCXGate, RCCCXGate
from qiskit.quantum_info import Operator

from zrl.utils import to_qiskit_ordering

ModuleNotFoundError: No module named 'zrl'

In [0]:
def _compute_control_matrix(base_mat, num_ctrl_qubits):
    """
    Compute the controlled version of the input matrix with qiskit ordering.

    Args:
        base_mat (ndarray): unitary to be controlled
        num_ctrl_qubits (int): number of controls for new unitary

    Returns:
        ndarray: controlled version of base matrix.
    """
    num_target = int(np.log2(base_mat.shape[0]))
    ctrl_dim = 2**num_ctrl_qubits
    ctrl_grnd = np.repeat([[1], [0]], [1, ctrl_dim-1])
    full_mat_dim = ctrl_dim * base_mat.shape[0]
    full_mat = np.zeros((full_mat_dim, full_mat_dim), dtype=base_mat.dtype)
    ctrl_proj = np.diag(np.roll(ctrl_grnd, ctrl_dim - 1))
    full_mat = (np.kron(np.eye(2**num_target),
                        np.eye(ctrl_dim) - ctrl_proj)
                + np.kron(base_mat, ctrl_proj))
    return full_mat

In [0]:
def _compute_control_matrix(base_mat, num_ctrl_qubits, ctrl_state=None):
    """
    Compute the controlled version of the input matrix with qiskit ordering.
    Args:
        base_mat (ndarray): unitary to be controlled
        num_ctrl_qubits (int): number of controls for new unitary
        ctrl_state (int or str or None): The control state in decimal or as
            a bitstring (e.g. '111'). If None, use 2**num_ctrl_qubits-1.
    Returns:
        ndarray: controlled version of base matrix.
    Raises:
        QiskitError: unrecognized mode or invalid ctrl_state
    """
    num_target = int(numpy.log2(base_mat.shape[0]))
    ctrl_dim = 2**num_ctrl_qubits
    ctrl_grnd = numpy.repeat([[1], [0]], [1, ctrl_dim-1])
    if ctrl_state is None:
        ctrl_state = ctrl_dim - 1
    elif isinstance(ctrl_state, str):
        ctrl_state = int(ctrl_state, 2)
    if isinstance(ctrl_state, int):
        if not 0 <= ctrl_state < ctrl_dim:
            raise QiskitError('Invalid control state value specified.')
    else:
        raise QiskitError('Invalid control state type specified.')
    full_mat_dim = ctrl_dim * base_mat.shape[0]
    full_mat = numpy.zeros((full_mat_dim, full_mat_dim), dtype=base_mat.dtype)
    ctrl_proj = numpy.diag(numpy.roll(ctrl_grnd, ctrl_state))
    full_mat = (numpy.kron(numpy.eye(2**num_target),
                           numpy.eye(ctrl_dim) - ctrl_proj)
                + numpy.kron(base_mat, ctrl_proj))
    return full_mat

In [0]:
B = np.eye(8, dtype=complex)
B[5, 5] = -1
B[6:, 6:] = np.array([[0, -1j], [1j, 0]])

CB = np.eye(16, 16, dtype=complex)
CB[8:, 8:] = B

cb = to_qiskit_ordering(CB)

In [0]:
plt.figure()
plt.imshow(np.abs(CB))
plt.figure()
plt.imshow(np.abs(cb))


In [0]:
ctrl = _compute_control_matrix(RCCXGate().to_matrix(), 1)
plt.imshow(np.abs(ctrl - cb))

In [0]:
num_ctrl_qubits = 1
for gate in [RCCXGate()]:
    cgate = gate.control(num_ctrl_qubits)
    base_mat = Operator(gate).data
    ctrl_mat = Operator(cgate).data
    target_mat = _compute_control_matrix(base_mat, num_ctrl_qubits)
    print(gate.name)

    diff = np.abs(ctrl_mat - target_mat)
    plt.figure()
    plt.imshow(np.abs(ctrl_mat))
    plt.colorbar()

    plt.figure()
    plt.imshow(diff)
    plt.colorbar()


In [0]:
cgate = RCCXGate().control()
qc = QuantumCircuit(cgate.num_qubits)
qc.append(cgate, list(range(cgate.num_qubits)), [])
qc.draw()

In [0]:
unitary = execute(qc, BasicAer.get_backend('unitary_simulator')).result().get_unitary()

In [0]:
plt.imshow(np.abs(unitary))
plt.imshow(np.abs(unitary - ctrl))

atol = 1e-10
for i in range(2**cgate.num_qubits):
    for j in range(2**cgate.num_qubits):
        if np.abs(ctrl[i, j] - unitary[i, j]) >= atol:
            print(i, j, ctrl[i, j], unitary[i, j])